# Restoring alphanumeric changes that the model did

The idea is to use the suggestions from the model, but only the ones which correct punctuation, apostrophes and spaces. These changes it seems to do well and the MT model can benefit from these changes alone (mainly as sentence tokenization).

I've compiled this code for playing with, with an edit distance function I took from the internet and a backtracing algorithm which tells what the changes were. Unfortunately, both are a little bit buggy so I found it hard to work, and they need to be fixed before going forward.

In [1]:
from __future__ import unicode_literals
from Levenshtein.StringMatcher import StringMatcher

In [12]:
origs = open('cassey/orig.txt').read().splitlines()
models = open('cassey/model.txt').read().splitlines()

In [13]:
len(origs) == len(models)

True

In [29]:
def minEditDist(target, source):

    # Length of the target strings set to variables
    n = len(target)
    m = len(source)

    distance = [[0 for i in range(m+1)] for j in range(n+1)]

    for i in range(1,n+1):
        distance[i][0] = distance[i-1][0] + 1

    for j in range(1,m+1):
        distance[0][j] = distance[0][j-1] + 1

    for i in range(1,n+1):
        for j in range(1,m+1):
            # 0 if there is a copy, 1 if there is a substitution
            sc = int(source[j-1] != target[i-1])
            distance[i][j] = min(distance[i-1][j]+1,
                                 distance[i][j-1]+1,
                                 distance[i-1][j-1]+sc)

    def backtrace(i, j):
        if i>0 and j>0 and distance[i-1][j-1] + 1 == distance[i][j]:
            return backtrace(i-1, j-1) + "S"
        if i>0 and j>0 and distance[i-1][j-1] == distance[i][j]:
            return backtrace(i-1, j-1) + "M"
        if i>0 and distance[i-1][j] + 1 == distance[i][j]:
            return backtrace(i-1, j) + "D"
        if j>0 and distance[i][j-1] + 1 == distance[i][j]:
            return backtrace(i, j-1) + "I"
        return ""

    # Return the minimum distance using all the table cells
    return distance[i][j], backtrace(i, j)


In [44]:
sent_orig = 'locatione is nice.friendly'
sent_model = 'location is nice. friednlie'
minEditDist(sent_orig, sent_model)

(6, u'IMMMMMMMMSSMMMMMMMMMMMMMSSS')

In [45]:
i = j = 0
orig = list(sent_orig)
model = list(sent_model)
for change in minEditDist(sent_orig, sent_model)[1]:
    print "orig: {} model: {} change: {}".format(orig[i], model[j], change)
    if change == 'M': # same character
        i += 1
        j += 1
    elif change == 'S': # substitution
        if orig[i].isalnum(): # changing of alphanumeric characters is discarded
            model[j] = orig[i]
        i += 1
        j += 1
    elif change == 'D': # deletion
        if orig[i].isalnum():
            model[j:j] = [orig[i]]
            j += 1
        else:
            i += 1
            j += 1
    elif change == 'I': # insertion
        if model[j].isalnum():
            del model[j]
            i += 1
        else:
            i += 1
            j += 1
print(''.join(orig))
print(''.join(model))

orig: l model: l change: I
orig: o model: o change: M
orig: c model: c change: M
orig: a model: a change: M
orig: t model: t change: M
orig: i model: i change: M
orig: o model: o change: M
orig: n model: n change: M
orig: e model:   change: M
orig:   model: i change: S
orig: i model: s change: S
orig: s model:   change: M
orig:   model: n change: M
orig: n model: i change: M
orig: i model: c change: M
orig: c model: e change: M
orig: e model: . change: M
orig: . model:   change: M
orig: f model: f change: M
orig: r model: r change: M
orig: i model: i change: M
orig: e model: e change: M
orig: n model: d change: M
orig: d model: n change: M
orig: l model: l change: S
orig: y model: i change: S


IndexError: list index out of range